In [ ]:
import rosetta
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
rosetta.init(extra_options = "-add_orbitals 1")
output_notebook()

In [12]:
pose = rosetta.core.pose.Pose()
rosetta.core.import_pose.pose_from_file(pose, "/Users/sheffler/1ffw_native.pdb")

core.chemical.ResidueTypeSet: Finished initializing fa_standard residue type set.  Created 387 residue types
core.chemical.ResidueTypeSet: Total time to initialize 0.480289 seconds.


In [13]:
sf = rosetta.get_fa_scorefxn()
opts = sf.energy_method_options()
opts.hbond_options().decompose_bb_hb_into_pair_energies(True)
opts.hbond_options().use_hb_env_dep(False)
sf.set_energy_method_options(opts)
sf.score(pose)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: talaris2014
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/rama/Rama_smooth_dyn.dat_ss_6.4
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA_n
basic.io.database

85.29013777910983

In [14]:
hbset = rosetta.core.scoring.hbonds.HBondSet()
rosetta.core.scoring.hbonds.fill_hbond_set(pose,False,hbset)
hbscore, hbdis = [],[]
for i in range(1,hbset.nhbonds()+1):
    hb = hbset.hbond(i)
#     if hb.energy() > -0.8:
#         continue
    rdon = pose.residue(hb.don_res())
    racc = pose.residue(hb.acc_res())
    ih = hb.don_hatm()
    ib = rdon.atom_base(ih)
    ia = hb.acc_atm()
    hname = rdon.atom_name(ih)
    bname = rdon.atom_name(ib)
    aname = racc.atom_name(ia)
    xyzbase = rdon.xyz(ih)
    xyzacc = racc.xyz(ia)
#     print "DON:", rdon.atom_name(ih), xyzbase, xyzbase.distance(xyzacc)
#     print "ACC:", racc.atom_name(ia), xyzacc
    hodists = [xyzbase.distance(racc.orbital_xyz(borb)) 
               for borb in racc.bonded_orbitals(ia)]
    for i, hodis in enumerate(hodists):
        if hodis == min(hodists):
            break
    io = racc.bonded_orbitals(ia)[i+1]
    xyzorb = racc.orbital_xyz(io)
    oname = racc.orbital_name(io)
#     print hb.energy(), hodis, bname, hname, oname, aname
    hbscore.append(hb.energy())
    hbdis.append(hodis)
#scoredist = np.zeros((len(hbdata),2))
#for i, (score, dist) in enumerate(hbdata):
#    scoredist[i,0] = score
#    scoredist[i,1] = dist


In [15]:
p = figure(title="hbond score vs o-h dist", toolbar_location=None)
p.scatter(hbdis, hbscore)
show(p)